# Running the pre-night briefing dashboard within a notebook

## Notebook perparation

### Load jupyter extensions

In [ ]:
%load_ext lab_black
%load_ext autoreload
%autoreload 1

### Imports

Use `aimport` for `schedview` imports for ease of debugging.

In [ ]:
import warnings
import math
import logging
from pathlib import Path
import panel as pn
import numpy as np
import pandas as pd
import param
import bokeh
from copy import deepcopy
import datetime
from pytz import timezone
import lzma
import pickle
import yaml
import json
from collections import OrderedDict
from tempfile import TemporaryDirectory, NamedTemporaryFile

In [ ]:
from astropy.time import Time, TimeDelta
from zoneinfo import ZoneInfo
import matplotlib as mpl
import matplotlib.pyplot as plt
import hvplot.pandas

In [ ]:
from rubin_sim.scheduler.example import example_scheduler
from rubin_sim.scheduler import sim_runner
from rubin_sim.scheduler.model_observatory import ModelObservatory
from rubin_sim.scheduler.utils import SchemaConverter

In [ ]:
%aimport schedview
%aimport schedview.app.prenight
%aimport schedview.compute.scheduler
%aimport schedview.collect.opsim
from schedview.plot.visitmap import BAND_COLORS

### Further preparation of the notebook

In [ ]:
pn.extension("terminal")

### Filter warnings

Several dependencies throw prodigious instances of (benign) warnings.
Suppress them to avoid poluting the executed notebook.

In [ ]:
warnings.filterwarnings(
    "ignore",
    module="astropy.time",
    message="Numerical value without unit or explicit format passed to TimeDelta, assuming days",
)
warnings.filterwarnings(
    "ignore",
    module="pandas",
    message="In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.",
)
warnings.filterwarnings(
    "ignore",
    module="healpy",
    message="divide by zero encountered in divide",
)
warnings.filterwarnings(
    "ignore",
    module="healpy",
    message="invalid value encountered in multiply",
)
warnings.filterwarnings(
    "ignore",
    module="holoviews",
    message="Discarding nonzero nanoseconds in conversion.",
)
warnings.filterwarnings(
    "ignore",
    module="rubin_sim",
    message="invalid value encountered in arcsin",
)
warnings.filterwarnings(
    "ignore",
    module="rubin_sim",
    message="All-NaN slice encountered",
)
warnings.filterwarnings(
    "ignore",
    module="rubin_sim.scheduler.utils",
    message="invalid value encountered in cast",
)

## Configuration and initial configuration

Setting `keep_rewards` to `True` results in a dashboard that includes plots of rewards.

In [ ]:
keep_rewards = True

Set the start date, scheduler, and observatory for the night:

In [ ]:
observatory = ModelObservatory()

Set `evening_mjd` to the integer calendar MJD of the local calendar day on which sunset falls on the night of interest.

In [ ]:
evening_iso8601 = "2025-01-01"

night_date = datetime.date.fromisoformat(evening_iso8601)
evening_mjd = Time(evening_iso8601).mjd
night_date, evening_mjd

If we just use this day as the start and make the simulation duration 1 day, the begin and end of the simulation will probably begin in the middle on one night and end in the middle of the next.
Instead, find the sunset and sunrise of the night we want using the almanac, and use these to determine our start time and duration.

In [ ]:
# If the date represents the local calendar date at sunset, we need to shift by the longitude in units of days
this_night = (
    np.floor(observatory.almanac.sunsets["sunset"] + observatory.site.longitude / 360)
    == evening_mjd
)

mjd_start = observatory.almanac.sunsets[this_night]["sun_n12_setting"][0]
mjd_end = observatory.almanac.sunsets[this_night]["sunrise"][0]

night_duration = mjd_end - mjd_start
time_start = Time(mjd_start, format="mjd")
time_start.iso, night_duration

In [ ]:
observatory = ModelObservatory(mjd_start=mjd_start)

In [ ]:
scheduler = example_scheduler(mjd_start=mjd_start)

Record the date of local day in the evening. 

## Run a simulation and create the app instance

For this example, simulate starting the default first day of observing:

In [ ]:
if not keep_rewards:
    observatory, scheduler, observations = sim_runner(
        observatory, scheduler, mjd_start=mjd_start, survey_length=night_duration
    )
else:
    scheduler.keep_rewards = True
    observatory, scheduler, observations, reward_df, obs_rewards = sim_runner(
        observatory,
        scheduler,
        mjd_start=mjd_start,
        survey_length=night_duration,
        record_rewards=True,
    )

## Save the simulation

In [ ]:
data_dir = TemporaryDirectory()

In [ ]:
with NamedTemporaryFile(prefix="opsim-", suffix=".db", dir=data_dir.name) as temp_file:
    opsim_output_fname = temp_file.name

SchemaConverter().obs2opsim(observations, filename=opsim_output_fname)
opsim_output_fname

In [ ]:
with NamedTemporaryFile(
    prefix="scheduler-", suffix=".pickle.xz", dir=data_dir.name
) as temp_file:
    scheduler_fname = temp_file.name

with lzma.open(scheduler_fname, "wb", format=lzma.FORMAT_XZ) as pio:
    pickle.dump(scheduler, pio)

scheduler_fname

In [ ]:
with NamedTemporaryFile(
    prefix="rewards-", suffix=".h5", dir=data_dir.name
) as temp_file:
    rewards_fname = temp_file.name

reward_df.to_hdf(rewards_fname, "reward_df")
obs_rewards.to_hdf(rewards_fname, "obs_rewards")

If you're host doesn't have a lot of memory, you may need to clean out some memory before trying to start the dashboard.

In [ ]:
# del observations
del scheduler
del reward_df
del obs_rewards

## Make some custom plots

Make some custom plots, trying them out in the notebook before we define a file with which they can be added to the dashboard.

Get the observations in the same form the custom plotter will see them:

In [ ]:
visits = schedview.collect.opsim.read_opsim(opsim_output_fname)

In [ ]:
hvplot_kwargs_slew = {
    "kind": "scatter",
    "x": "slewDistance",
    "y": "slewTime",
    "ylabel": "slew time (seconds)",
    "xlabel": "slew distance (degrees)",
    "color": "visitTime",
    "size": "visitExposureTime",
    "clabel": "visit time (seconds)",
    "cmap": "isolum",
}
visits.hvplot(**hvplot_kwargs_slew)

In [ ]:
hvplot_kwargs_airmass_hist = {
    "kind": "hist",
    "y": "airmass",
    "by": "note",
    "bins": 15,
    "height": 512,
    "ylabel": "number of visits",
}
visits.hvplot(**hvplot_kwargs_airmass_hist)

Put these settings into json file that can be read by the dashboard:

In [ ]:
with NamedTemporaryFile(
    prefix="custom_prenight_tabs-", suffix=".json", dir=data_dir.name
) as temp_file:
    custom_tabs_fname = temp_file.name

with open(custom_tabs_fname, "w") as custom_tabs_file:
    custom_json = json.dump(
        [
            {"name": "Slew Time", "settings": hvplot_kwargs_slew},
            {"name": "Airmass histogram", "settings": hvplot_kwargs_airmass_hist},
        ],
        indent=4,
        fp=custom_tabs_file,
    )

Read it back to look at the contents:

In [ ]:
with open(custom_tabs_fname, "r") as custom_tabs_file:
    custom_json = custom_tabs_file.read()

print(custom_json)

## Make the dashboard

Including two instances of the scheduler takes too much memory, crashes the kernel. Bummer.

In [ ]:
prenight = schedview.app.prenight.Prenight()
pn_app = prenight.make_app(
    night_date,
    opsim_db=opsim_output_fname,
    scheduler=scheduler_fname,
    rewards=rewards_fname,
    custom_hvplot_tab_settings_file=custom_tabs_fname,
)

In [ ]:
show_inline = True

if show_inline:
    out = pn_app
else:
    out = "Show with panel button at top of jupyter tab"

out

# Exploring the data, and extracting settings that can be used to specify new plots

You can use the `hvplot` explorer tool to investigate the visits table beyond what is in the dashboard:

Use the explorer GUI above to get the plot as close as you can to what you want, and get the settings for use in a custom plot json file (as described above) thus:

## Creating a custom dashboard with more complex custom plots

The intended appreach for adding plots that cannot be achieved through `hvplot` to a dashboard is to:

1. Create a python function that returns a plot or other figure that can be displayed by `panel`. This can be anything that panel can put in a pane, including matplotlib plots, bokeh and holoviews plots, png or gif images, or even other panel elements. See [this page in the panel documenation](https://panel.holoviz.org/reference/index.html#panes) for examples.
2. Subclass `schedview.app.prenight.prenight.Prenight`, adding a methods to call your new plotting function and return its result, and expand the `schedview.app.prenight.prenight.Prenight.initialize_tab_contents` method to including your new plot.

For examples and more detailed instructions, start with the `prenight_matplotlib_externsion.ipynb` notebook, and then the `prenight_multielements_extension.ipynb` notebook.